In [2]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import tensorboard
from sentence_transformers import SentenceTransformer

import hdbscan 
import plotly.express as px
import nbformat

sys.path.append("/Users/alina/Desktop/MIT/code/ADHD/item_analysis")
from src.topic_modeling import *

In [3]:
data_root = Path("/Volumes/Samsung_T5/MIT/abcd/release_05/abcd-data-release-5.1")
vocab_path  = Path("/Volumes/Samsung_T5/MIT/abcd/adhd_vocab.npy")

section = "core/mental-health"
table = 'mh_p_ksads_adhd.csv'
mapping = 'ksads_adhd.csv'

data_section = Path( data_root, section)
data_path = Path(data_section, table) # path to results table  
var_mapping_path =  Path(data_root, "var_mapping", mapping) # path to mapping of variable names to questions 

data = pd.read_csv(data_path)
var_description = pd.read_csv(var_mapping_path)
vocab = np.load(vocab_path, allow_pickle=True)

var_names = np.array(var_description["var_name"])
questions = np.array(var_description["var_label"])
mapping = OrderedDict(zip(var_names, questions))

to_map = questions


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_91690/3472203376.py:12: DtypeWarning: Columns (23,76,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


In [16]:
# embeddings param 
model = "avsolatorio/GIST-Embedding-v0"

# dimensionality reduction param
n_neighbors = 15
n_components = 3
min_dist = 0.0
metric = "cosine"


 #cluster params 
algorithm = "best"
min_cluster_size = 2
gen_min_span_tree = True
metric = "euclidean"
cluster_selection_method = "eom"

In [17]:

sentence_embedding_model = SentenceTransformer(model)       # load embedding
sentence_embedding_model._first_module().max_seq_length = 500 # if you want to increase, but makes it slower
embeddings = sentence_embedding_model.encode(to_map, convert_to_tensor=True,show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/117k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
umap_data = dimensionality_reduction(embeddings, 
                                        n_neighbors=n_neighbors, 
                                        n_components=n_components, 
                                        min_dist=min_dist, 
                                        metric='cosine'
                                        )

In [19]:
cluster = hdbscan.HDBSCAN(algorithm= algorithm, 
                          min_cluster_size=min_cluster_size,
                          gen_min_span_tree = gen_min_span_tree,
                          metric= metric,       
                          cluster_selection_method = cluster_selection_method).fit(umap_data)

In [20]:
if n_components==3:
    result = pd.DataFrame(umap_data, columns=['x', 'y', 'z'])
elif n_components==2:
        result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_
result["questions"] = to_map

In [21]:
for qst in result.questions:
    print(qst)

In the past two weeks, how often has your child put off or avoided doing things that were hard for him or her because they required a lot of attention?
Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family
Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With friends
In the past two weeks, how often has your child talked when he or she wasn't supposed to, or had people complain that he or she talked too much?
In the past two weeks, how often has your child interrupted someone when they were talking?
In the past two weeks, how often has your child had trouble waiting for his or her turn or waiting for other things?
In the past two weeks, how often has your child blurt out answers in school without being called on, or blurt out answers when someone wasn't talking to him or her?
In the past two weeks, how often has your child had trouble being quiet when he 

In [22]:
outliers = result.loc[(result.labels == '-1'), :]
clustered = result.loc[(result.labels != '-1'), :]

In [23]:
if n_components==2:
    fig = px.scatter(clustered, 'x', 'y', color="labels", hover_data="questions") # size='Size',
elif n_components==3:
    fig = px.scatter_3d(clustered, x='x', y='y', z='z', color=result.labels, hover_data="questions")

In [24]:
fig.show()